In [5]:
# Type greedy-style algorithm. Similar to Core DM Algorithm-v3 
# Program reads an external data file.  
# Solve the 3-IAP problem without sorting matrices. 
# In Tie cases, programs picks a minimum at random. 
# Repeat running more than once to obtain better solutions. Updated 21 Sept. 2021.  

import sys
import numpy as np
#from pynput import keyboard
import pandas as pd
import math
import random
import time
from scipy.optimize import linear_sum_assignment
from statistics import mean, median, pstdev

start_time = time.time()

num_list = []

def load_data(file_name):
    #a = np.loadtxt(r"C:\Users\mehbalim\Desktop\Temp Sep 2021\B&S_3IAP_n4_instance1.txt")  
    a = np.loadtxt(file_name)
    a = list(a)
    a = [int(i) for i in a]

    b = len(a)
    n = math.pow(b, 1./3.)
    if type(n) != "int":
        n = math.ceil(n)

    for x in a:
        x = int(x)
        num_list.append(x)

    factory =[]
    m = n*n
    for j in range(n):
        lb=j*m
        ub=(j+1)*m 
        factory = factory + num_list[lb:ub]

    factories = [factory[j:j+m] for j in range(0, len(factory), m)]
    #print('factories = ',factories) 
    return (a, factories)
 

data_file = r"C:\Users\mehbalim\Desktop\Temp Sep 2021\Random Sample\Rand_3IAP_n55_instance1.txt"
#data_file = 'C:\\Users\\mehbalim\\Rand_3IAP_n65_instance1.txt'
(a, factories) = load_data(data_file)
n = len(factories)

assign = {'Jobs':[],'Machines':[],'Factories':[],'Cost':[]}
df = pd.DataFrame(assign)
df = df.astype('int64', copy=False)
df["Cost"] = df['Cost'].astype('float')

tables = factories

#print()
print("The 3-Index Assignment Problem is of size", n, ", i.e.", n, "jobs for", n, "machines, in", n, "factories. \n")

def select_One_minimum(matrix, k, option = 0):
    """
    This function selects one min candidate from a list of candidates, i.e. tie-case.
    """

    row_ind, col_ind = linear_sum_assignment(matrix)
    cost_list = matrix[(row_ind, col_ind)]

    #print("Assignment of minimum cost for this matrix is ") 
    #print(cost_list, " The value of the minimum cost is ", min(cost_list), '.\n')
    
    minlyst=[]
    for i in range(k):
        if cost_list[i] == min(cost_list):
            minlyst.append([row_ind[i], col_ind[i]])
    #print(minlyst, '\n')

    min_indices = [i for i in range(k) if cost_list[i] == min(cost_list)]
     
    if option == 1: ## random
    #if len(min_indices) > 1:  # Tie cases!    
        mi = len(min_indices)
        #print('The minimum cost occurs :', mi, 'times, then the problem contains a Tie case. \n')
        #t1 = time.time()
        #print('before the input t1 = ', t1)
        #
        t=(random.randint(0,mi-1))  
        #print(t)
        row_ix=min_indices[t]
    else:#first or last (i.e. 0 or -1)
        row_ix=min_indices[option]
        
    #global cost_value
    cost_value=min(cost_list)
    
    row_indx = -1
    col_indx = -1
    for j in range(n):
        if j==row_ix:
            #global row_indx, col_indx
            row_indx=row_ind[j]
            col_indx=col_ind[j]
            #print("The coordinates of the minimum selected are : index_row =", row_indx, "and index_col =", col_indx,'\n')
            break  
    
    return (cost_value, row_indx, col_indx)

def two_matrices(mat):
    fact0=mat[0]
    fact1=mat[1]
    lst=[]
    #print("The two matrices are :", '\n')
    #print("Fact1 = ", fact0, '\n')
    #print("Fact2 = ", fact1, '\n')
    
    for i in range(4):
        v=fact0[i]+fact1[3-i]
        lst.append(v)
        
    #print("List of candidate costs = ", lst,'\n')
    #global opt_cost, cost0 , cost1
    opt_cost = min(lst) 
    ##print("Optimal cost = ", opt_cost,'\n')
    #print("Index of Optimal cost = ", lst.index(opt_cost))
    
    #global ix, indx, jndx, kndx
    ix=lst.index(opt_cost)
    
    cost0=fact0[ix] 
    cost1=fact1[3-ix]
                           
    if ix==0:
        indx=0
        jndx=0
    
    elif ix==1:
        indx=0
        jndx=1

    elif ix==2:
        indx=1
        jndx=0
 
    else:
        indx=1
        jndx=1
    kndx=n-2  
     
    #print("Optimal cost = ", opt_cost, " or ", cost0, " + ", cost1, '.\n') 
   
    #global assign0, assign1
    assign0={'Jobs':indx, 'Machines':jndx,'Factories':kndx,'Cost':cost0}
    assign1={'Jobs':1-indx, 'Machines':1-jndx,'Factories':kndx+1,'Cost':cost1}
    
    return(assign0, assign1, opt_cost)

k = n
factnb=-1
Total = 0.00
while k!=2:
    newT = []
    i = 0
    #print("The value of k=", k, '\n')
    for t in tables:
        if i == 0:
            arr=np.asarray(t).reshape(k,k)
            arr = np.array(arr)
            #print(arr, '\n')
            (cost_value, row_indx, col_indx) = select_One_minimum(arr, k, option = 1)
            factnb=factnb+1
            
            assign={'Jobs':row_indx, 'Machines':col_indx,'Factories':factnb,'Cost':cost_value}
            df=df.append(assign, ignore_index=True)
            df = df.astype('int64', copy=False)
            Total=Total + cost_value
            
            new_table=[]
            ix=row_indx
            iy=col_indx
        else:
            #print("Reduce the matrice ",i)
            arr=np.asarray(t).reshape(k,k)
            #print(arr, '\n')
            row_idx = ix
            col_idx = iy
            new_mat = np.delete(arr, row_idx, 0)
            new_mat = np.delete(new_mat, col_idx, 1)
            #print(new_mat,'\n')
            new_mat = np.array(new_mat).flatten().tolist()
            newT.append(new_mat)
        i=i+1
    k=k-1
    tables=newT
    #print(k,"matrices", tables, '\n')
    
(assign0, assign1, opt_cost) = two_matrices(tables)
Total=Total + opt_cost
df=df.append(assign0, ignore_index=True)
df=df.append(assign1, ignore_index=True)
df = df.astype('int64', copy=False)
      
#print(df,'\n')

print("Total cost = ", "%.2f" % Total,'\n')

print("--- %.8f seconds ---" % (time.time() - start_time))


The 3-Index Assignment Problem is of size 55 , i.e. 55 jobs for 55 machines, in 55 factories. 

Total cost =  601.00 

--- 1.13942575 seconds ---
